In [1]:
#| default_exp 34_metadexa-for-msmarco

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

In [24]:
#| export
import os,torch,json, torch.multiprocessing as mp, joblib, numpy as np, scipy.sparse as sp

from transformers import DistilBertConfig

from xcai.basics import *
from xcai.models.oakY import OAK008

In [5]:
os.environ['WANDB_MODE'] = 'disabled'

In [6]:
#| export
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
os.environ['WANDB_PROJECT'] = 'mogicX_00-msmarco'

## Setup

In [7]:
output_dir = '/scratch/scai/phd/aiz218323/outputs/mogicX/34_metadexa-for-msmarco'
pkl_dir = '/scratch/scai/phd/aiz218323/datasets/processed/mogicX'

config_file = '/scratch/scai/phd/aiz218323/datasets/msmarco/XC/configs/entity_gpt_exact.json'
config_key = 'data_entity-gpt_exact'

mname = 'sentence-transformers/msmarco-distilbert-cos-v5'

In [8]:
do_train_inference = False
do_test_inference = False

save_train_inference = False
save_test_inference = False

save_representation = False

use_sxc_sampler, only_test = True, False

In [9]:
pkl_file = get_pkl_file(pkl_dir, 'msmarco_data-meta_distilbert-base-uncased', use_sxc_sampler)

In [10]:
pkl_file

'/scratch/scai/phd/aiz218323/datasets/processed/mogicX/msmarco_data-meta_distilbert-base-uncased_sxc.joblib'

In [11]:
do_inference = do_train_inference or do_test_inference or save_train_inference or save_test_inference or save_representation

In [12]:
%%time
os.makedirs(os.path.dirname(pkl_file), exist_ok=True)
block = build_block(pkl_file, config_file, use_sxc_sampler, config_key, do_build=False, only_test=False, meta_oversample=True, 
                    meta_dropout_remove=0.3, meta_dropout_replace=0.3)


CPU times: user 16.2 s, sys: 1.89 s, total: 18.1 s
Wall time: 18.2 s


In [13]:
meta_name = 'ent'

In [30]:
batch = block.train.dset.__getitems__([10, 20, 50])

{k:v for k,v in batch.items() if f'{meta_name}2' in k}

{'pent2data_idx': tensor([  490,   113, 11483]),
 'pent2data_data2ptr': tensor([1, 1, 1]),
 'ent2data_idx': tensor([  490,   113, 11483]),
 'ent2data_dropout_remove_mask': tensor([False,  True,  True]),
 'ent2data_dropout_replace_mask': tensor([ True, False, False]),
 'ent2data_data2ptr': tensor([1, 1, 1]),
 'ent2data_identifier': ['490', '113', '11483'],
 'ent2data_input_text': ['cars', 'mountain', 'venus'],
 'ent2data_input_ids': tensor([[ 101, 3079,  102,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0],
         [ 101, 3231,  102,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0],
         [ 101, 1396, 9299,  102,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    

In [25]:
model = OAK008.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4', batch_size=100, num_batch_labels=5000, 
                               margin=0.3, num_negatives=10, tau=0.1, apply_softmax=True,
                               
                               data_aug_meta_prefix=f'{meta_name}2data', lbl2data_aug_meta_prefix=None, 
                               data_pred_meta_prefix=None, lbl2data_pred_meta_prefix=None,

                               num_metadata=block.train.dset.meta[f'{meta_name}_meta'].n_meta, num_metadata_clusters=block.train.dset.meta[f'{meta_name}_meta'].n_meta,
                               
                               calib_margin=0.3, calib_num_negatives=10, calib_tau=0.1, calib_apply_softmax=True, 
                               calib_loss_weight=0.1, use_calib_loss=True,

                               use_query_loss=True,
                               
                               use_encoder_parallel=False, do_meta_embed_sparse=False)
model.init_retrieval_head()
model.init_meta_encoder()
model.init_cross_head()

Some weights of OAK008 were not initialized from the model checkpoint at sentence-transformers/msmarco-distilbert-base-v4 and are newly initialized: ['encoder.alpha', 'encoder.cross_head.k.bias', 'encoder.cross_head.k.weight', 'encoder.cross_head.o.bias', 'encoder.cross_head.o.weight', 'encoder.cross_head.q.bias', 'encoder.cross_head.q.weight', 'encoder.cross_head.v.bias', 'encoder.cross_head.v.weight', 'encoder.dr_head.layer_norm.bias', 'encoder.dr_head.layer_norm.weight', 'encoder.dr_head.projector.bias', 'encoder.dr_head.projector.weight', 'encoder.dr_head.transform.bias', 'encoder.dr_head.transform.weight', 'encoder.meta_distilbert.embeddings.LayerNorm.bias', 'encoder.meta_distilbert.embeddings.LayerNorm.weight', 'encoder.meta_distilbert.embeddings.position_embeddings.weight', 'encoder.meta_distilbert.embeddings.word_embeddings.weight', 'encoder.meta_distilbert.transformer.layer.0.attention.k_lin.bias', 'encoder.meta_distilbert.transformer.layer.0.attention.k_lin.weight', 'encoder.

In [34]:
b = prepare_batch(model, batch, m_args=[
    f'p{meta_name}2data_idx', f'p{meta_name}2data_data2ptr', f'{meta_name}2data_idx', f'{meta_name}2data_input_ids', 
    f'{meta_name}2data_attention_mask', f'{meta_name}2data_data2ptr', f'{meta_name}2data_dropout_remove_mask', 
    f'{meta_name}2data_dropout_replace_mask'
])

In [27]:
model = model.to('cuda')

In [28]:
o = model(**b.to(model.device))

/scratch/scai/phd/aiz218323/projects/xcai/xcai/losses.py:22: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /opt/conda/conda-bld/pytorch_1724789115564/work/aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  return torch.sparse_csr_tensor(data_ptr, data_idx, scores, device=data_ptr.device)


In [32]:
def func():
    import pdb; pdb.set_trace()
    o = model(**b.to(model.device))
    

In [ ]:
func()

> /tmp/ipykernel_7783/3721260802.py(3)func()
      1 def func():
      2     import pdb; pdb.set_trace()
----> 3     o = model(**b.to(model.device))
      4 



ipdb>  c


> /scratch/scai/phd/aiz218323/projects/xcai/xcai/models/oakY.py(827)forward()
    825         **kwargs
    826     ):  
--> 827         return_dict = return_dict if return_dict is not None else self.config.use_return_dict
    828 
    829         if self.use_encoder_parallel:



ipdb>  n


> /scratch/scai/phd/aiz218323/projects/xcai/xcai/models/oakY.py(829)forward()
    827         return_dict = return_dict if return_dict is not None else self.config.use_return_dict
    828 
--> 829         if self.use_encoder_parallel:
    830             encoder = XCDataParallel(module=self.encoder)
    831         else: encoder = self.encoder



...

> /scratch/scai/phd/aiz218323/projects/xcai/xcai/models/oakY.py(1127)replace_dropout()
   1125 
   1126         if mask is None: x[idx] = 0
-> 1127         else: x[valid_row_idx[idx]] = 0
   1128 
   1129     @staticmethod



ipdb>  


--Return--
None
> /scratch/scai/phd/aiz218323/projects/xcai/xcai/models/oakY.py(1127)replace_dropout()
   1125 
   1126         if mask is None: x[idx] = 0
-> 1127         else: x[valid_row_idx[idx]] = 0
   1128 
   1129     @staticmethod



ipdb>  n


> /scratch/scai/phd/aiz218323/projects/xcai/xcai/models/oakY.py(1145)_get_encoder_meta_kwargs()
   1143                     self.remove_dropout(meta_repr, remove_mask)
   1144                     self.replace_dropout(meta_repr, self.metadata_dropout, mask=replace_mask)
-> 1145                 meta_kwargs[f'{prefix}_meta_repr'] = meta_repr
   1146         return meta_kwargs
   1147 



In [37]:
args = XCLearningArguments(
    output_dir=output_dir,
    logging_first_step=True,
    per_device_train_batch_size=512,
    per_device_eval_batch_size=512,
    representation_num_beams=200,
    representation_accumulation_steps=10,
    save_strategy="steps",
    eval_strategy="steps",
    eval_steps=5000,
    save_steps=5000,
    save_total_limit=5,
    num_train_epochs=300,
    predict_with_representation=True,
    adam_epsilon=1e-6,
    warmup_steps=100,
    weight_decay=0.01,
    learning_rate=2e-4,
    representation_search_type='BRUTEFORCE',

    output_representation_attribute='data_fused_repr',
    label_representation_attribute='data_repr',
    metadata_representation_attribute='data_repr',
    data_augmentation_attribute='data_repr',
    representation_attribute='data_fused_repr',
    clustering_representation_attribute='data_fused_repr',

    group_by_cluster=True,
    num_clustering_warmup_epochs=10,
    num_cluster_update_epochs=5,
    num_cluster_size_update_epochs=25,
    use_data_metadata_for_clustering=True,
    clustering_type='EXPO',
    minimum_cluster_size=2,
    maximum_cluster_size=1600,

    metric_for_best_model='P@1',
    load_best_model_at_end=True,
    target_indices_key='plbl2data_idx',
    target_pointer_key='plbl2data_data2ptr',

    use_distributional_representation=False,
    use_encoder_parallel=True,
    max_grad_norm=None,
    fp16=True,
    
    label_names=['plbl2data_idx', 'plbl2data_data2ptr', 'lbl2data_idx', 'lbl2data_data2ptr', 'lbl2data_input_ids', 'lbl2data_attention_mask', 
                 f'{meta_name}2data_idx', f'{meta_name}2data_data2ptr', f'{meta_name}2data_input_ids', f'{meta_name}2data_attention_mask', 
                 f'{meta_name}2data_dropout_remove_mask', f'{meta_name}2data_dropout_replace_mask'],

    prune_metadata=False,
    num_metadata_prune_warmup_epochs=10,
    num_metadata_prune_epochs=5,
    metadata_prune_batch_size=2048,
    prune_metadata_names=[f'{meta_name}_meta'],
    use_data_metadata_for_pruning=True,

    predict_with_augmentation=False,
    use_augmentation_index_representation=True,

    data_aug_meta_name=meta_name,
    augmentation_num_beams=None,
    data_aug_prefix=meta_name,
    use_label_metadata=False,

    data_meta_batch_size=2048,
    augment_metadata=False,
    num_metadata_augment_warmup_epochs=10,
    num_metadata_augment_epochs=5,

    use_cpu_for_searching=True,
    use_cpu_for_clustering=True,
)

In [39]:
learn = XCLearner(
    model=model,
    args=args,
    train_dataset=block.train.dset,
    eval_dataset=block.test.dset,
    data_collator=block.collator,
)

Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [40]:
trn_dl = learn.get_train_dataloader()
batch = next(iter(trn_dl))

In [41]:
def func():
    import pdb; pdb.set_trace()
    o = model(**batch.to(model.device))
    

In [43]:
func()

## Driver

In [ ]:
#| export
if __name__ == '__main__':
    output_dir = '/scratch/scai/phd/aiz218323/outputs/mogicX/34_metadexa-for-msmarco'

    config_file = '/scratch/scai/phd/aiz218323/datasets/msmarco/XC/configs/entity_gpt_exact.json'
    config_key = 'data_entity-gpt_exact'

    meta_name = 'ent'
    
    mname = 'sentence-transformers/msmarco-distilbert-cos-v5'

    input_args = parse_args()

    pkl_file = get_pkl_file(f'{input_args.pickle_dir}/mogicX', 'msmarco_data_distilbert-base-uncased', input_args.use_sxc_sampler, 
                            input_args.exact)

    do_inference = input_args.do_train_inference or input_args.do_test_inference or input_args.save_train_prediction or input_args.save_test_prediction or input_args.save_representation

    os.makedirs(os.path.dirname(pkl_file), exist_ok=True)
    block = build_block(pkl_file, config_file, input_args.use_sxc_sampler, config_key, do_build=input_args.build_block, 
                        only_test=input_args.only_test, n_slbl_samples=3, main_oversample=False, n_sdata_meta_samples=3, 
                        meta_oversample=False, train_meta_topk=5, test_meta_topk=3, meta_dropout_remove=0.3, 
                        meta_dropout_replace=0.3)

    args = XCLearningArguments(
        output_dir=output_dir,
        logging_first_step=True,
        per_device_train_batch_size=512,
        per_device_eval_batch_size=512,
        representation_num_beams=200,
        representation_accumulation_steps=10,
        save_strategy="steps",
        eval_strategy="steps",
        eval_steps=5000,
        save_steps=5000,
        save_total_limit=5,
        num_train_epochs=300,
        predict_with_representation=True,
        adam_epsilon=1e-6,
        warmup_steps=100,
        weight_decay=0.01,
        learning_rate=2e-4,
        representation_search_type='BRUTEFORCE',
    
        output_representation_attribute='data_fused_repr',
        label_representation_attribute='data_repr',
        metadata_representation_attribute='data_repr',
        data_augmentation_attribute='data_repr',
        representation_attribute='data_fused_repr',
        clustering_representation_attribute='data_fused_repr',

        group_by_cluster=True,
        num_clustering_warmup_epochs=10,
        num_cluster_update_epochs=5,
        num_cluster_size_update_epochs=25,
        use_data_metadata_for_clustering=True,
        clustering_type='EXPO',
        minimum_cluster_size=2,
        maximum_cluster_size=1600,

        metric_for_best_model='P@1',
        load_best_model_at_end=True,
        target_indices_key='plbl2data_idx',
        target_pointer_key='plbl2data_data2ptr',
    
        use_distributional_representation=False,
        use_encoder_parallel=True,
        max_grad_norm=None,
        fp16=True,
        
        label_names=['plbl2data_idx', 'plbl2data_data2ptr', 'lbl2data_idx', 'lbl2data_data2ptr', 'lbl2data_input_ids', 'lbl2data_attention_mask', 
                     f'{meta_name}2data_idx', f'{meta_name}2data_data2ptr', f'{meta_name}2data_input_ids', f'{meta_name}2data_attention_mask', 
                     f'{meta_name}2data_dropout_remove_mask', f'{meta_name}2data_dropout_replace_mask'],
        
        prune_metadata=False,
        num_metadata_prune_warmup_epochs=10,
        num_metadata_prune_epochs=5,
        metadata_prune_batch_size=2048,
        prune_metadata_names=[f'{meta_name}_meta'],
        use_data_metadata_for_pruning=True,
    
        predict_with_augmentation=False,
        use_augmentation_index_representation=True,

        data_aug_meta_name=meta_name,
        augmentation_num_beams=None,
        data_aug_prefix=meta_name,
        use_label_metadata=False,

        data_meta_batch_size=2048,
        augment_metadata=False,
        num_metadata_augment_warmup_epochs=10,
        num_metadata_augment_epochs=5,
    
        use_cpu_for_searching=True,
        use_cpu_for_clustering=True,
    )
    
    def model_fn(mname, bsz):
        model = OAK008.from_pretrained(mname, batch_size=bsz, num_batch_labels=5000, 
                                       margin=0.3, num_negatives=10, tau=0.1, apply_softmax=True,
                               
                                       data_aug_meta_prefix=f'{meta_name}2data', lbl2data_aug_meta_prefix=None, 
                                       data_pred_meta_prefix=None, lbl2data_pred_meta_prefix=None,
        
                                       num_metadata=block.train.dset.meta[f'{meta_name}_meta'].n_meta, 
                                       num_metadata_clusters=block.train.dset.meta[f'{meta_name}_meta'].n_meta,

                                       calib_loss_weight=0.1, use_calib_loss=True,
                                       calib_margin=0.05, calib_num_negatives=10, calib_tau=0.1, calib_apply_softmax=False, 
        
                                       use_query_loss=True,
                                       
                                       use_encoder_parallel=True, do_meta_embed_sparse=False, metadata_dropout=0.3)
        return model
    
    def init_fn(model):
        model.init_retrieval_head()
        model.init_meta_encoder()
        model.init_cross_head()
        model.init_meta_embeddings()

    metric = PrecReclMrr(block.n_lbl, block.test.data_lbl_filterer, prop=block.train.dset.data.data_lbl,
                         pk=10, rk=200, rep_pk=[1, 3, 5, 10], rep_rk=[10, 100, 200], mk=[5, 10, 20])

    bsz = max(args.per_device_train_batch_size, args.per_device_eval_batch_size)*torch.cuda.device_count()
    model = load_model(args.output_dir, model_fn, {"mname": mname, "bsz": bsz}, init_fn, do_inference=do_inference, use_pretrained=input_args.use_pretrained)
    
    learn = XCLearner(
        model=model,
        args=args,
        train_dataset=block.train.dset,
        eval_dataset=block.test.dset,
        data_collator=block.collator,
        compute_metrics=metric,
    )
    
    main(learn, input_args, n_lbl=block.test.dset.n_lbl, eval_k=10, train_k=10)
    